In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import tlib

sns.set_style("whitegrid")

plt.rcParams["text.usetex"] = True
plt.rcParams["font.family"] = "serif"
plt.rcParams["font.size"] = "10"
plt.rcParams["text.latex.preamble"] = r"\boldmath"


# Borrowed from https://stackoverflow.com/a/31527287/2102808
def get_figsize(columnwidth, wf=0.5, hf=(5.**0.5-1.0)/2.0, ):
    """Parameters:
    - wf [float]:  width fraction in columnwidth units
    - hf [float]:  height fraction in columnwidth units.
                   Set by default to golden ratio.
    - columnwidth [float]: width of the column in latex. Get this from LaTeX 
                           using \showthe\columnwidth
    Returns:  [fig_width,fig_height]: that should be given to matplotlib
    """
    fig_width_pt = columnwidth*wf 
    inches_per_pt = 1.0/72.27               # Convert pt to inch
    fig_width = fig_width_pt*inches_per_pt  # width in inches
    fig_height = fig_width*hf      # height in inches
    return [fig_width, fig_height]

def coherent_clusters(df, col, value=None):
    clusters = []
    current = pd.DataFrame(df[:0])
    prev = None
    
    for index, row in df.iterrows():        
        if prev is None:
            prev = row[col]
        
        if prev == row[col]:
            if value is not None:
                if row[col] == value:
                    current.loc[index, :] = row
            else:
                current.loc[index, :] = row
        elif len(current) > 0:
            clusters.append(current.copy())
            current = current[:0]

        prev = row[col]
    
    if len(current) > 0:
        clusters.append(current.copy())
    
    return clusters

df = tlib.read_tfw2018_1()
df["Time"] = pd.to_datetime(df["Time"])

dates = sorted(pd.unique([timestamp.strftime("%Y-%m-%d") for timestamp in df[df["EVENT"]]["Time"]]))
index = 7
start = pd.Timestamp(dates[index])
end = start + pd.DateOffset(hours=24)

print(start, end)

span = df[(df["Time"] >= start) & (df["Time"] <= end)].copy()
clusters = coherent_clusters(span, "EVENT", True)
cols = ["Tp", "Cl", "pH", "Redox", "Leit", "Trueb", "Cl_2", "Fm", "Fm_2"]

for i, col in enumerate(cols):
    plt.figure(figsize=get_figsize(345, wf=1.0, hf=0.25), dpi=600)
    ax = sns.lineplot(span, x="Time", y=col)
    ax.set(xlabel=r"\Large{\textbf{Time}}", ylabel=r"\Large{\textbf{" + col + r"}")
    
    # Set the desired number of x-axis ticks
    num_ticks = 13

    # Get the data range for x-axis
    x_min = span["Time"].min()
    x_max = span["Time"].max()
    # Get the data range for y-axis
    y_min = span[col].min()
    y_max = span[col].max()
    y_diff = y_max - y_min
    y_off = 0.1 * y_diff

    # Generate evenly spaced tick positions
    tick_positions = pd.date_range(start=x_min, end=x_max, periods=num_ticks)

    # Format the tick labels as desired (e.g., '%Y-%m-%d %H:%M:%S')
    tick_labels = [timestamp.strftime("%H:%M") for timestamp in tick_positions]

    # Set the custom tick positions and labels on the x-axis
    ax.set_xticks(tick_positions)
    ax.set_xticklabels(tick_labels, rotation=45)
    
    for cluster in clusters:
        ax = sns.lineplot(cluster, x="Time", y=col, color="red")
 
    ax.fill_between(x=span['Time'], y1=y_min-y_off, y2=y_max+y_off, where=(span["EVENT"]), color="red", alpha=0.3)  # Adjust color and alpha as needed
    
    plt.savefig(f"fin_01_data_set_{i}_{col}.png", bbox_inches="tight")